Group 1 Google Colab environment.

Upgrade our pip environment to ensure we have access to the latest pip packages on pypi.

In [2]:
!python3 -m pip install --upgrade pip

Install dependencies

In [3]:
!python3 -m pip install \
datasets \
implicit \
scikit-surprise

Import our required packages

In [4]:
import gzip
import random
import scipy
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from datasets import load_dataset
from implicit import bpr
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

Load the Amazon reviews dataset from Huggingface

In [5]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Musical_Instruments", trust_remote_code=True)
print(dataset["full"][0])

#Print our splits
print(dataset.keys())
print(len(dataset["full"]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'rating': 5.0, 'title': 'Five Stars', 'text': 'Great headphones, comfortable and sound is good. No complaints. I would buy again.', 'images': [], 'asin': 'B003LPTAYI', 'parent_asin': 'B003LPTAYI', 'user_id': 'AGKASBHYZPGTEPO6LWZPVJWB2BVA', 'timestamp': 1452650586000, 'helpful_vote': 0, 'verified_purchase': True}
dict_keys(['full'])
3017439


Load the dataset into a pandas dataframe

In [14]:
df = pd.DataFrame(dataset['full'][:len(dataset['full']) // 10]).sample(frac=0.5)
print(df.head())

print(df.dtypes)
reader = Reader(rating_scale=(1, 5))

surprise_data = Dataset.load_from_df(df[['title', 'text', 'rating']], reader)


        rating                                              title  \
129202     4.0    It worked for what was intended. They are a ...   
255033     3.0                            How about some outputs?   
111975     5.0  Affordable baritone saxophone that won't disap...   
20510      5.0                         Sturdy..good sound quality   
89485      3.0                                        Three Stars   

                                                     text  \
129202  It worked for what was intended. They are a bi...   
255033                     USB only has 2 channels. Fail.   
111975  Jay Metcalf  brought me here also, and man am ...   
20510   A lot of fun. Learning as I go..fun for easing...   
89485                           Good for intended purpose   

                                                   images        asin  \
129202                                                 []  B004H6PC4M   
255033                                                 []  B07XG6PPMT   

In [16]:
model = SVD()
trainset, testset = train_test_split(surprise_data, test_size=.25)

model.fit(trainset)
predictions = model.test(testset)

In [19]:
predictions[0].est

4.608400874540823

In [18]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

1.1169467206169177
